In [17]:
import sqlite3
import re
import unicodedata
import nltk
from nltk.corpus import stopwords
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# Baixar stopwords portuguesas
nltk.download('stopwords')
stop_words_pt = set(stopwords.words('portuguese'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vitor\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [18]:
import sqlite3
import re

def limpar_texto(texto):
    """
    Limpa o texto para melhorar o treinamento.
    Remove quebras de linha, espaços extras e caracteres especiais.
    """
    # Remove quebras de linha e tabs
    texto = texto.replace('\n', ' ').replace('\r', ' ').replace('\t', ' ')
    # Remove múltiplos espaços
    texto = re.sub(r'\s+', ' ', texto)
    # Remove caracteres especiais indesejados, mantendo pontuação básica
    texto = re.sub(r'[^a-zA-ZÀ-ÿ0-9,.!?;:()\'" ]', '', texto)
    # Remove espaços no início e fim
    texto = texto.strip()
    return texto

In [19]:
def remover_stopwords(texto):
    """
    Remove stop words do texto em português.
    """
    texto = texto.lower()
    palavras = [p for p in texto.split() if p not in stop_words_pt]
    return ' '.join(palavras)

In [20]:
def limpar_e_atualizar_banco(db_path="textos.db"):
    """
    Limpa todos os textos no banco, cria coluna 'texto_semSW' e atualiza os registros.
    """
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    # Adiciona coluna 'texto_semSW' se não existir
    try:
        cursor.execute("ALTER TABLE textos ADD COLUMN texto_semSW TEXT")
    except sqlite3.OperationalError:
        print("Coluna 'texto_semSW' já existe.")

    # Seleciona todos os textos com seus IDs
    cursor.execute("SELECT id, texto FROM textos")
    registros = cursor.fetchall()

    for registro_id, texto in registros:
        texto_limpo = limpar_texto(texto)
        texto_sem_sw = remover_stopwords(texto_limpo)
        # Atualiza o registro no banco
        cursor.execute(
            "UPDATE textos SET texto = ?, texto_semSW = ? WHERE id = ?",
            (texto_limpo, texto_sem_sw, registro_id)
        )

    conn.commit()
    conn.close()
    print(f"{len(registros)} textos foram limpos e atualizados no banco com stop words removidas.")


In [ ]:
# Exemplo de uso
limpar_e_atualizar_banco()

In [21]:
def gerar_nuvem_palavras_autores(db_path="textos.db"):
    """
    Gera uma nuvem de palavras para cada autor a partir dos textos sem stopwords.
    """
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    cursor.execute("SELECT DISTINCT autor FROM textos")
    autores = [row[0] for row in cursor.fetchall()]

    for autor in autores:
        cursor.execute("SELECT texto_semSW FROM textos WHERE autor = ?", (autor,))
        textos = " ".join([row[0] for row in cursor.fetchall()])

        wordcloud = WordCloud(width=800, height=400, background_color='white').generate(textos)

        plt.figure(figsize=(10, 5))
        plt.imshow(wordcloud, interpolation='bilinear')
        plt.title(f'Nuvem de Palavras - {autor}')
        plt.axis('off')
        plt.savefig(f'{autor}_wordcloud.png')
        plt.close()

    conn.close()
    print("Nuvens de palavras geradas para cada autor.")

# %%
gerar_nuvem_palavras_autores()

Nuvens de palavras geradas para cada autor.
